# 0.导入相关模块并定义全局变量

In [1]:
from os import listdir
from os.path import isfile,join
import re 
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_columns', 200)

path = r'/home/duhan/data/商品期货1分钟201509csv'
filenamePattern = r'(?P<code>\D+)(?P<num>\d+).csv' 
header=['date', 'time', 'open', 'high', 'low', 'close', 'lot', 'volume']

# 1.读取目录中的所有文件名列表

In [2]:
filenameList = [ filename for filename in listdir(path) if isfile(join(path,filename))  and re.match(filenamePattern, filename)]
filenameDataset = DataFrame(filenameList, columns=['name'])
filenameDataset['code'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('code'))
filenameDataset['num'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('num'))
filenameDataset.head()

,name,code,num
0,ZZOI13.csv,ZZOI,13
1,DLJM12.csv,DLJM,12
2,DLV13.csv,DLV,13
3,SQAG01.csv,SQAG,01
4,DLCS05.csv,DLCS,05


# 2. 获取所有品种列表

In [3]:
instrumentList  = list(filenameDataset.groupby('code').count().index)
print instrumentList
filenameDataset.groupby('code').count().head(10)

['DLAX', 'DLAY', 'DLB', 'DLBB', 'DLC', 'DLCS', 'DLFB', 'DLI', 'DLJ', 'DLJD', 'DLJM', 'DLL', 'DLM', 'DLP', 'DLPP', 'DLV', 'DLY', 'SQAG', 'SQAL', 'SQAU', 'SQBUX', 'SQBUY', 'SQCU', 'SQFU', 'SQHC', 'SQNI', 'SQPB', 'SQRB', 'SQRU', 'SQSN', 'SQWR', 'SQZN', 'ZZCF', 'ZZFG', 'ZZJR', 'ZZLR', 'ZZMA', 'ZZME', 'ZZOI', 'ZZPM', 'ZZRI', 'ZZRM', 'ZZRS', 'ZZSF', 'ZZSM', 'ZZSRX', 'ZZSRY', 'ZZTA', 'ZZTC', 'ZZWH']


,name,num
code,,
DLAX,7,7
DLAY,4,4
DLB,5,5
DLBB,5,5
DLC,7,7
DLCS,6,6
DLFB,7,7
DLI,13,13
DLJ,11,11


# 3. 定义通过文名件导入数据集的方法

In [4]:
def loadDataFile(filename):
    """
    根据文件名导入数据
    """
    num = re.match(filenamePattern, filename).group('num')
    df =  pd.read_csv( join(path, filename), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'], x['time']) , axis =1)
    df.columns = [ [num]*8, df.columns ]
    return df


In [5]:
loadDataFile('DLAX05.csv').head()

05                                          
                        date   time  open  high   low close lot volume
2015-09-01 09:01  2015-09-01  09:01  4307  4307  4305  4307  24  33522
2015-09-01 09:02  2015-09-01  09:02  4304  4305  4303  4305  36  33512
2015-09-01 09:03  2015-09-01  09:03  4303  4303  4303  4303  16  33502
2015-09-01 09:04  2015-09-01  09:04  4302  4302  4299  4299  38  33498
2015-09-01 09:05  2015-09-01  09:05  4299  4299  4295  4296  56  33494

# 4.定义导入一种品种所有数据的函数

In [6]:
def loadInstrumentData(code):
    """
    给定顶一个品种代码，导入该品种的所有数据文件
    """
    c1 = filenameDataset['code'] == code
    df = pd.concat([ loadDataFile(filename) for filename in filenameDataset[ c1 ]['name'] ] , axis=1)    
    df = df.swaplevel(0,1,axis=1)
    df = df.sort_index(axis=1)
    return df

In [7]:
df =loadInstrumentData('DLI').head(10)
df 

close                                                      \
                     01  02     03   04     05     06  07  08     09     10   
2015-09-01 09:01  388.0 NaN    NaN  NaN  367.0  363.5 NaN NaN  466.0    NaN   
2015-09-01 09:02  387.5 NaN    NaN  NaN  367.0    NaN NaN NaN  465.5  423.0   
2015-09-01 09:03  389.0 NaN    NaN  NaN  368.0    NaN NaN NaN  464.5    NaN   
2015-09-01 09:04  388.5 NaN    NaN  NaN  368.5    NaN NaN NaN  464.5    NaN   
2015-09-01 09:05  390.5 NaN    NaN  NaN  369.0    NaN NaN NaN  465.0    NaN   
2015-09-01 09:06  390.0 NaN  371.0  NaN  369.5    NaN NaN NaN  466.0    NaN   
2015-09-01 09:07  392.0 NaN  372.5  NaN  371.0    NaN NaN NaN  468.0    NaN   
2015-09-01 09:08  391.0 NaN    NaN  369  370.5    NaN NaN NaN  467.5  437.5   
2015-09-01 09:09  392.5 NaN    NaN  NaN  372.0    NaN NaN NaN  467.5    NaN   
2015-09-01 09:10  392.5 NaN    NaN  NaN  371.5    NaN NaN NaN  467.5    NaN   

                                          date                               \
                  11     12     13          01   02          03          04   
2015-09-01 09:01 NaN    NaN  385.6  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:02 NaN    NaN  385.2  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:03 NaN    NaN  386.6  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:04 NaN    NaN  386.3  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:05 NaN  392.5  388.1  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:06 NaN    NaN  387.7  2015-09-01  NaN  2015-09-01         NaN   
2015-09-01 09:07 NaN    NaN  389.5  2015-09-01  NaN  2015-09-01         NaN   
2015-09-01 09:08 NaN    NaN  389.1  2015-09-01  NaN         NaN  2015-09-01   
2015-09-01 09:09 NaN    NaN  389.7  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:10 NaN    NaN  389.7  2015-09-01  NaN         NaN         NaN   

                                                                            \
                          05          06   07   08          09          10   
2015-09-01 09:01  2015-09-01  2015-09-01  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:02  2015-09-01         NaN  NaN  NaN  2015-09-01  2015-09-01   
2015-09-01 09:03  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:04  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:05  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:06  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:07  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:08  2015-09-01         NaN  NaN  NaN  2015-09-01  2015-09-01   
2015-09-01 09:09  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:10  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   

                                                high                         \
                   11          12          13     01  02     03   04     05   
2015-09-01 09:01  NaN         NaN  2015-09-01  389.0 NaN    NaN  NaN  368.5   
2015-09-01 09:02  NaN         NaN  2015-09-01  388.0 NaN    NaN  NaN  367.5   
2015-09-01 09:03  NaN         NaN  2015-09-01  389.0 NaN    NaN  NaN  368.0   
2015-09-01 09:04  NaN         NaN  2015-09-01  389.0 NaN    NaN  NaN  368.5   
2015-09-01 09:05  NaN  2015-09-01  2015-09-01  390.5 NaN    NaN  NaN  369.0   
2015-09-01 09:06  NaN         NaN  2015-09-01  390.5 NaN  371.5  NaN  369.5   
2015-09-01 09:07  NaN         NaN  2015-09-01  392.5 NaN  372.5  NaN  371.0   
2015-09-01 09:08  NaN         NaN  2015-09-01  392.0 NaN    NaN  369  371.0   
2015-09-01 09:09  NaN         NaN  2015-09-01  392.5 NaN    NaN  NaN  372.0   
2015-09-01 09:10  NaN         NaN  2015-09-01  393.0 NaN    NaN  NaN  372.5   

                                                                   lot      \
                     06  07  08     09     10  11     12     13     01  02   
2015-09-01 09:01  363.5 NaN NaN  466.0    NaN NaN    NaN  386.7  25296 NaN   


 # 5. 计算加权平均过程

In [8]:
df =loadInstrumentData('DLI').head(10)
df 

close                                                      \
                     01  02     03   04     05     06  07  08     09     10   
2015-09-01 09:01  388.0 NaN    NaN  NaN  367.0  363.5 NaN NaN  466.0    NaN   
2015-09-01 09:02  387.5 NaN    NaN  NaN  367.0    NaN NaN NaN  465.5  423.0   
2015-09-01 09:03  389.0 NaN    NaN  NaN  368.0    NaN NaN NaN  464.5    NaN   
2015-09-01 09:04  388.5 NaN    NaN  NaN  368.5    NaN NaN NaN  464.5    NaN   
2015-09-01 09:05  390.5 NaN    NaN  NaN  369.0    NaN NaN NaN  465.0    NaN   
2015-09-01 09:06  390.0 NaN  371.0  NaN  369.5    NaN NaN NaN  466.0    NaN   
2015-09-01 09:07  392.0 NaN  372.5  NaN  371.0    NaN NaN NaN  468.0    NaN   
2015-09-01 09:08  391.0 NaN    NaN  369  370.5    NaN NaN NaN  467.5  437.5   
2015-09-01 09:09  392.5 NaN    NaN  NaN  372.0    NaN NaN NaN  467.5    NaN   
2015-09-01 09:10  392.5 NaN    NaN  NaN  371.5    NaN NaN NaN  467.5    NaN   

                                          date                               \
                  11     12     13          01   02          03          04   
2015-09-01 09:01 NaN    NaN  385.6  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:02 NaN    NaN  385.2  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:03 NaN    NaN  386.6  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:04 NaN    NaN  386.3  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:05 NaN  392.5  388.1  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:06 NaN    NaN  387.7  2015-09-01  NaN  2015-09-01         NaN   
2015-09-01 09:07 NaN    NaN  389.5  2015-09-01  NaN  2015-09-01         NaN   
2015-09-01 09:08 NaN    NaN  389.1  2015-09-01  NaN         NaN  2015-09-01   
2015-09-01 09:09 NaN    NaN  389.7  2015-09-01  NaN         NaN         NaN   
2015-09-01 09:10 NaN    NaN  389.7  2015-09-01  NaN         NaN         NaN   

                                                                            \
                          05          06   07   08          09          10   
2015-09-01 09:01  2015-09-01  2015-09-01  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:02  2015-09-01         NaN  NaN  NaN  2015-09-01  2015-09-01   
2015-09-01 09:03  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:04  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:05  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:06  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:07  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:08  2015-09-01         NaN  NaN  NaN  2015-09-01  2015-09-01   
2015-09-01 09:09  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   
2015-09-01 09:10  2015-09-01         NaN  NaN  NaN  2015-09-01         NaN   

                                                high                         \
                   11          12          13     01  02     03   04     05   
2015-09-01 09:01  NaN         NaN  2015-09-01  389.0 NaN    NaN  NaN  368.5   
2015-09-01 09:02  NaN         NaN  2015-09-01  388.0 NaN    NaN  NaN  367.5   
2015-09-01 09:03  NaN         NaN  2015-09-01  389.0 NaN    NaN  NaN  368.0   
2015-09-01 09:04  NaN         NaN  2015-09-01  389.0 NaN    NaN  NaN  368.5   
2015-09-01 09:05  NaN  2015-09-01  2015-09-01  390.5 NaN    NaN  NaN  369.0   
2015-09-01 09:06  NaN         NaN  2015-09-01  390.5 NaN  371.5  NaN  369.5   
2015-09-01 09:07  NaN         NaN  2015-09-01  392.5 NaN  372.5  NaN  371.0   
2015-09-01 09:08  NaN         NaN  2015-09-01  392.0 NaN    NaN  369  371.0   
2015-09-01 09:09  NaN         NaN  2015-09-01  392.5 NaN    NaN  NaN  372.0   
2015-09-01 09:10  NaN         NaN  2015-09-01  393.0 NaN    NaN  NaN  372.5   

                                                                   lot      \
                     06  07  08     09     10  11     12     13     01  02   
2015-09-01 09:01  363.5 NaN NaN  466.0    NaN NaN    NaN  386.7  25296 NaN   


In [9]:
df['close']

,01,02,03,04,05,06,07,08,09,10,11,12,13
2015-09-01 09:01,388.0,NaN,NaN,NaN,367.0,363.5,NaN,NaN,466.0,NaN,NaN,NaN,385.6
2015-09-01 09:02,387.5,NaN,NaN,NaN,367.0,NaN,NaN,NaN,465.5,423.0,NaN,NaN,385.2
2015-09-01 09:03,389.0,NaN,NaN,NaN,368.0,NaN,NaN,NaN,464.5,NaN,NaN,NaN,386.6
2015-09-01 09:04,388.5,NaN,NaN,NaN,368.5,NaN,NaN,NaN,464.5,NaN,NaN,NaN,386.3
2015-09-01 09:05,390.5,NaN,NaN,NaN,369.0,NaN,NaN,NaN,465.0,NaN,NaN,392.5,388.1
2015-09-01 09:06,390.0,NaN,371.0,NaN,369.5,NaN,NaN,NaN,466.0,NaN,NaN,NaN,387.7
2015-09-01 09:07,392.0,NaN,372.5,NaN,371.0,NaN,NaN,NaN,468.0,NaN,NaN,NaN,389.5
2015-09-01 09:08,391.0,NaN,NaN,369,370.5,NaN,NaN,NaN,467.5,437.5,NaN,NaN,389.1
2015-09-01 09:09,392.5,NaN,NaN,NaN,372.0,NaN,NaN,NaN,467.5,NaN,NaN,NaN,389.7
2015-09-01 09:10,392.5,NaN,NaN,NaN,371.5,NaN,NaN,NaN,467.5,NaN,NaN,NaN,389.7


In [10]:
df['lot'] 

,01,02,03,04,05,06,07,08,09,10,11,12,13
2015-09-01 09:01,25296,NaN,NaN,NaN,1662,2,NaN,NaN,6,NaN,NaN,NaN,27006
2015-09-01 09:02,3084,NaN,NaN,NaN,146,NaN,NaN,NaN,10,2,NaN,NaN,3202
2015-09-01 09:03,7920,NaN,NaN,NaN,994,NaN,NaN,NaN,4,NaN,NaN,NaN,8922
2015-09-01 09:04,1692,NaN,NaN,NaN,396,NaN,NaN,NaN,8,NaN,NaN,NaN,2092
2015-09-01 09:05,29378,NaN,NaN,NaN,2086,NaN,NaN,NaN,40,NaN,NaN,10,31514
2015-09-01 09:06,17754,NaN,4,NaN,2424,NaN,NaN,NaN,204,NaN,NaN,NaN,20386
2015-09-01 09:07,48900,NaN,10,NaN,5148,NaN,NaN,NaN,62,NaN,NaN,NaN,54164
2015-09-01 09:08,32942,NaN,NaN,2,4188,NaN,NaN,NaN,50,2,NaN,NaN,37154
2015-09-01 09:09,21792,NaN,NaN,NaN,5788,NaN,NaN,NaN,2,NaN,NaN,NaN,27588
2015-09-01 09:10,35434,NaN,NaN,NaN,2054,NaN,NaN,NaN,22,NaN,NaN,NaN,37528


In [11]:
df['lot'].sum( axis = 1)

2015-09-01 09:01     53972
2015-09-01 09:02      6444
2015-09-01 09:03     17840
2015-09-01 09:04      4188
2015-09-01 09:05     63028
2015-09-01 09:06     40772
2015-09-01 09:07    108284
2015-09-01 09:08     74338
2015-09-01 09:09     55170
2015-09-01 09:10     75038
dtype: float64

In [12]:
r = df['lot'].div(df['lot'].sum( axis = 1), axis=0)
r

,01,02,03,04,05,06,07,08,09,10,11,12,13
2015-09-01 09:01,0.468687,NaN,NaN,NaN,0.030794,0.000037,NaN,NaN,0.000111,NaN,NaN,NaN,0.500371
2015-09-01 09:02,0.478585,NaN,NaN,NaN,0.022657,NaN,NaN,NaN,0.001552,0.000310,NaN,NaN,0.496896
2015-09-01 09:03,0.443946,NaN,NaN,NaN,0.055717,NaN,NaN,NaN,0.000224,NaN,NaN,NaN,0.500112
2015-09-01 09:04,0.404011,NaN,NaN,NaN,0.094556,NaN,NaN,NaN,0.001910,NaN,NaN,NaN,0.499522
2015-09-01 09:05,0.466110,NaN,NaN,NaN,0.033096,NaN,NaN,NaN,0.000635,NaN,NaN,0.000159,0.500000
2015-09-01 09:06,0.435446,NaN,0.000098,NaN,0.059453,NaN,NaN,NaN,0.005003,NaN,NaN,NaN,0.500000
2015-09-01 09:07,0.451590,NaN,0.000092,NaN,0.047542,NaN,NaN,NaN,0.000573,NaN,NaN,NaN,0.500203
2015-09-01 09:08,0.443138,NaN,NaN,0.000027,0.056337,NaN,NaN,NaN,0.000673,0.000027,NaN,NaN,0.499798
2015-09-01 09:09,0.394997,NaN,NaN,NaN,0.104912,NaN,NaN,NaN,0.000036,NaN,NaN,NaN,0.500054
2015-09-01 09:10,0.472214,NaN,NaN,NaN,0.027373,NaN,NaN,NaN,0.000293,NaN,NaN,NaN,0.500120


In [13]:
df['close']  * r

,01,02,03,04,05,06,07,08,09,10,11,12,13
2015-09-01 09:01,181.850737,NaN,NaN,NaN,11.301304,0.01347,NaN,NaN,0.051805,NaN,NaN,NaN,192.942889
2015-09-01 09:02,185.451583,NaN,NaN,NaN,8.315022,NaN,NaN,NaN,0.722377,0.131285,NaN,NaN,191.404469
2015-09-01 09:03,172.695067,NaN,NaN,NaN,20.504036,NaN,NaN,NaN,0.104148,NaN,NaN,NaN,193.343341
2015-09-01 09:04,156.958453,NaN,NaN,NaN,34.843840,NaN,NaN,NaN,0.887297,NaN,NaN,NaN,192.965521
2015-09-01 09:05,182.016072,NaN,NaN,NaN,12.212572,NaN,NaN,NaN,0.295107,NaN,NaN,0.062274,194.050000
2015-09-01 09:06,169.823899,NaN,0.036398,NaN,21.967723,NaN,NaN,NaN,2.331600,NaN,NaN,NaN,193.850000
2015-09-01 09:07,177.023383,NaN,0.034400,NaN,17.637952,NaN,NaN,NaN,0.267962,NaN,NaN,NaN,194.829134
2015-09-01 09:08,173.266997,NaN,NaN,0.009928,20.872959,NaN,NaN,NaN,0.314442,0.011771,NaN,NaN,194.471487
2015-09-01 09:09,155.036433,NaN,NaN,NaN,39.027297,NaN,NaN,NaN,0.016948,NaN,NaN,NaN,194.871191
2015-09-01 09:10,185.344026,NaN,NaN,NaN,10.168994,NaN,NaN,NaN,0.137064,NaN,NaN,NaN,194.896740


In [14]:
close = (df['close']  * r).sum(axis=1)
close

2015-09-01 09:01    386.160205
2015-09-01 09:02    386.024736
2015-09-01 09:03    386.646592
2015-09-01 09:04    385.655110
2015-09-01 09:05    388.636025
2015-09-01 09:06    388.009619
2015-09-01 09:07    389.792832
2015-09-01 09:08    388.947583
2015-09-01 09:09    388.951869
2015-09-01 09:10    390.546824
dtype: float64

# 6.定义指标计算函数

In [15]:
def calculateIndex(df):
    """
    通过一个品种的数据全集计算其加权平均
    """
    calColumns = ['open', 'high', 'low', 'close']
    r =  df['lot'].div(df['lot'].sum( axis = 1), axis=0)
    resultList = [ (df[col]  * r).sum(axis=1) for col in calColumns ]
    result = pd.concat(resultList, axis=1, keys = calColumns)    
    return result


In [16]:
calculateIndex(df)    

,open,high,low,close
2015-09-01 09:01,386.225639,387.225491,385.725713,386.160205
2015-09-01 09:02,386.264804,386.524581,386.024736,386.024736
2015-09-01 09:03,385.646928,386.646704,385.224798,386.646592
2015-09-01 09:04,385.960649,386.057880,385.557880,385.655110
2015-09-01 09:05,387.218963,388.636025,386.719360,388.636025
2015-09-01 09:06,388.397665,388.427391,387.482444,388.009619
2015-09-01 09:07,388.342612,390.268729,387.842944,389.792832
2015-09-01 09:08,389.590976,389.669125,388.669515,388.947583
2015-09-01 09:09,387.899449,389.201896,387.701950,388.951869
2015-09-01 09:10,390.310717,391.310424,390.310717,390.546824


# 7. 定义完整的计算一个品种的加权平均加并保存到文件

In [17]:
def processInstrument(code):
    """
    计算品种的加权平均价，并保存到文件
    """
    df = loadInstrumentData(code)
    result = calculateIndex(df)
    filename = 'calculate_index_%s.csv' % code
    result.to_csv(join(path, filename), header=False)

In [18]:
processInstrument('DLI')

# 8. 循环处理所有品种

In [19]:
for code in instrumentList:
    print code,'...........',
    processInstrument(code)
    print u'完成'

DLAX ........... 完成
DLAY ........... 完成
DLB ........... 完成
DLBB ........... 完成
DLC ........... 完成
DLCS ........... 完成
DLFB ........... 完成
DLI ........... 完成
DLJ ........... 完成
DLJD ........... 完成
DLJM ........... 完成
DLL ........... 完成
DLM ........... 完成
DLP ........... 完成
DLPP ........... 完成
DLV ........... 完成
DLY ........... 完成
SQAG ........... 完成
SQAL ........... 完成
SQAU ........... 完成
SQBUX ........... 完成
SQBUY ........... 完成
SQCU ........... 完成
SQFU ........... 完成
SQHC ........... 完成
SQNI ........... 完成
SQPB ........... 完成
SQRB ........... 完成
SQRU ........... 完成
SQSN ........... 完成
SQWR ........... 完成
SQZN ........... 完成
ZZCF ........... 完成
ZZFG ........... 完成
ZZJR ........... 完成
ZZLR ........... 完成
ZZMA ........... 完成
ZZME ........... 完成
ZZOI ........... 完成
ZZPM ........... 完成
ZZRI ........... 完成
ZZRM ........... 完成
ZZRS ........... 完成
ZZSF ........... 完成
ZZSM ........... 完成
ZZSRX ........... 完成
ZZSRY ........... 完成
ZZTA ........... 完成
ZZTC ........... 完成
ZZWH ........... 完成
